In [ ]:
import tensorflow as tf

print(tf.__version__)


In [ ]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
from tqdm import tqdm_notebook, tnrange
from glob import glob
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
import keras

import tensorflow as tf
from skimage.color import rgb2gray
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, add,LeakyReLU
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator




In [ ]:
import pandas as pd

csv_file_path = r'C:\Users\vishnu\Downloads\monocular depth estimation\depth estimation\nyu_data\nyu2_train.csv'

image_filenames = []
mask_filenames = []


df = pd.read_csv(csv_file_path)
x = 0
for index, row in df.iterrows():
    if x == 4000:
        break
    x = x + 1
    image_path = row[0]  
    depth_path = row[1]    
    

    image_filenames.append("C:/Users/vishnu/Downloads/monocular depth estimation/depth estimation/nyu_data/"+ image_path)
    mask_filenames.append("C:/Users/vishnu/Downloads/monocular depth estimation/depth estimation/nyu_data/"+ depth_path)



In [ ]:
print(image_filenames[5])
print(mask_filenames[5])

In [ ]:

print("Image List:")
print(len(image_filenames))
print("\nDepth List:")
print(len(mask_filenames))

In [ ]:
def plot_image_mask(img_path, mask_path):
    image = cv2.imread(img_path)
    mask = cv2.imread(mask_path)

    if image is None or mask is None:
        raise ValueError("Error")

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_rgb = cv2.resize(image_rgb,(256,256))
    mask_rgb = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
    mask_rgb = cv2.resize(mask_rgb,(256,256))

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(image_rgb)
    axs[1].imshow(mask_rgb)
    plt.tight_layout()
    plt.show()

    

In [ ]:
plot_image_mask(image_filenames[343], mask_filenames[343])

In [ ]:
df = pd.DataFrame(data = {"images": image_filenames, "masks": mask_filenames})

In [ ]:
print(df)

In [ ]:
df_train, df_test = train_test_split(df, test_size = 0.1)

df_train, df_val = train_test_split(df_train, test_size = 0.2)

In [ ]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)


In [ ]:
def normalize(image,mask):
    image = image / 255
    mask = mask / 255
    return (image, mask) 


In [ ]:
def trainGenerator(
    data_frame,
    batch_size,
    augumentation_dict = {},
    image_color_mode = "rgb",
    mask_color_mode = "grayscale",
    image_save_prefix = "aug-img",
    mask_save_prefix = "aug-masks",
    save_to_dir = None,
    target_size = (256,256),
    seed = 1,
):
    image_datagen = ImageDataGenerator(**augumentation_dict)
    mask_datagen = ImageDataGenerator(**augumentation_dict)
    image_generator = image_datagen.flow_from_dataframe(
        data_frame,
        x_col = "images",
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed
    )
    mask_generator = mask_datagen.flow_from_dataframe(
        data_frame,
        x_col = "masks",
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed
     )
    
    train_gen = zip(image_generator,mask_generator)
    for image,mask in train_gen:
        image,mask = normalize(image,mask)
        yield (image,mask)
    

In [ ]:
def dice_coefficients(y_true,y_pred, smooth = 100):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)
    
    intersection = K.sum(y_true_flatten * y_pred_flatten)
    union = K.sum(y_true_flatten) + K.sum(y_pred_flatten)
    
    return (2 * intersection + smooth) / (union + smooth)

In [ ]:
def dice_coefficients_loss(y_true,y_pred,smooth = 100):
    return -dice_coefficients(y_true,y_pred,smooth)
    

In [ ]:
def intersection_over_union(y_true,y_pred,smooth = 100):
    intersection = K.sum(y_true * y_pred)
    sum = K.sum(y_true + y_pred)
    iou = (intersection + smooth) / (sum - intersection + smooth)
    return iou

In [ ]:
def jaccard_distance(y_true,y_pred):
    y_true_flatten = K.flatten(y_true)
    y_pred_flatten = K.flatten(y_pred)
    
    return -intersection_over_union(y_true_flatten,y_pred_flatten)

In [ ]:
def depth_loss_function(y_true, y_pred, theta=0.1, maxDepthVal=100):
    
    # Point-wise depth
    l_depth = K.mean(K.abs(y_pred - y_true), axis=-1)

    # Edges
    dy_true, dx_true = tf.image.image_gradients(y_true)
    dy_pred, dx_pred = tf.image.image_gradients(y_pred)
    l_edges = K.mean(K.abs(dy_pred - dy_true) + K.abs(dx_pred - dx_true), axis=-1)

    # Structural similarity (SSIM) index
    l_ssim = K.clip((1 - tf.image.ssim(y_true, y_pred, maxDepthVal)) * 0.5, 0, 1)

    # Weights
    w1 = 1.0
    w2 = 1.0
    w3 = theta

    return (w1 * l_ssim) + (w2 * K.mean(l_edges)) + (w3 * K.mean(l_depth))

In [ ]:
from tensorflow.keras.layers import Input, concatenate, LeakyReLU, BatchNormalization, Conv2D, MaxPooling2D, Dropout, UpSampling2D
def downsampling_block(input_tensor, n_filters):
  x = Conv2D(filters=n_filters, kernel_size=(3,3), padding='same')(input_tensor)
  x = LeakyReLU(alpha=0.2)(x)
  x = BatchNormalization()(x)

  x = Conv2D(filters=n_filters, kernel_size=(3,3), padding='same')(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = BatchNormalization()(x)
  return x

def upsampling_block(input_tensor, n_filters, name, concat_with):
  x = UpSampling2D((2, 2), interpolation='bilinear', name=name)(input_tensor)
  x = Conv2D(filters=n_filters, kernel_size=(3, 3), padding='same', name=name+"_convA")(x)
  x = LeakyReLU(alpha=0.2)(x)

  x = concatenate([x, concat_with], axis=3)

  x = Conv2D(filters=n_filters, kernel_size=(3, 3), padding='same', name=name+"_convB")(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = BatchNormalization()(x)

  x = Conv2D(filters=n_filters, kernel_size=(3, 3), padding='same', name=name+"_convC")(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = BatchNormalization()(x)
  return x

def build(height, width, depth):
  # input
  i = Input(shape=(height, width, depth))

  # encoder
  conv1 = downsampling_block(i, 32)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  conv2 = downsampling_block(pool1, 64)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  conv3 = downsampling_block(pool2, 128)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
  conv4 = downsampling_block(pool3, 256)
  pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

  # bottleneck
  conv5 = Conv2D(512, (3, 3), padding='same')(pool4)
  conv5 = LeakyReLU(alpha=0.2)(conv5)
  conv5 = Conv2D(512, (3, 3), padding='same')(conv5)
  conv5 = LeakyReLU(alpha=0.2)(conv5)

  # decoder
  conv6 = upsampling_block(conv5, 256, "up1", concat_with=conv4)
  conv7 = upsampling_block(conv6, 128, "up2", concat_with=conv3)
  conv8 = upsampling_block(conv7, 64, "up3", concat_with=conv2)
  conv9 = upsampling_block(conv8, 32, "up4", concat_with=conv1)

  # output
  o = Conv2D(filters=1, kernel_size=3, strides=(1,1), activation='sigmoid', padding='same', name='conv10')(conv9)

  model = Model(inputs=i, outputs=o)
  return model

In [ ]:
model = build(256,256,3)
model.summary()

In [ ]:
image_height =256
image_width = 256
EPOCHS = 15
BATCH_SIZE = 32
smooth = 100
learning_rate = 0.01

In [ ]:
def depth_loss(y_true, y_pred):
  w1, w2, w3 = 1.0, 1.0, 0.1

  l_depth = K.mean(K.abs(y_pred - y_true), axis=-1)

  dy_true, dx_true = tf.image.image_gradients(y_true)
  dy_pred, dx_pred = tf.image.image_gradients(y_pred)
  l_edges = K.mean(K.abs(dy_pred - dy_true) + K.abs(dx_pred - dx_true), axis=-1)

  l_ssim = K.clip((1 - tf.image.ssim(y_true, y_pred, 1.0)) * 0.5, 0, 1)

  return (w1 * l_ssim) + (w2 * K.mean(l_edges)) + (w3 * K.mean(l_depth))

#custom soft accuracy
def depth_acc(y_true, y_pred):
  return K.mean(K.equal(K.round(y_true), K.round(y_pred)))

In [ ]:
train_generator_param = dict(rotation_range=0.2,
                              width_shift_range= 0.05,
                              height_shift_range = 0.05,
                              shear_range = 0.05,
                              zoom_range = 0.05,
                              horizontal_flip = True,
                              fill_mode = 'nearest')
train_gen = trainGenerator(df_train,BATCH_SIZE,train_generator_param)
    
test_gen = trainGenerator(df_val,BATCH_SIZE)
    


initial_learning_rate = 0.001
decay_steps = 10000
decay_rate = learning_rate / EPOCHS

learning_rate_schedule = ExponentialDecay(
    initial_learning_rate, decay_steps, decay_rate, staircase=True
)

optimizer = Adam( beta_1 = 0.9,beta_2 = 0.999,learning_rate=learning_rate_schedule,amsgrad = True)
    
model.compile(optimizer = optimizer, loss= depth_loss, metrics = [depth_acc])

callbacks = [ModelCheckpoint('depth_estimation.hdf5',verbose = 1, save_best_only = True)]
    
history = model.fit(train_gen,
                    steps_per_epoch=len(df_train)/BATCH_SIZE,
                    epochs = EPOCHS,
                    callbacks = callbacks,
                    validation_data = test_gen,
                    validation_steps = len(df_val) / BATCH_SIZE
)    

In [ ]:
# model = load_model('C:/Users/vishnu/Downloads/skincancer/unet.hdf5', custom_objects = custom_objects)
custom_objects = {
    "depth_loss":depth_loss,
    "depth_acc":depth_acc
}
with keras.saving.custom_object_scope(custom_objects):
    reconstructed_model = keras.models.load_model("depth_estimation.hdf5")

In [ ]:
print(custom_objects)

In [ ]:
test_gen = trainGenerator(df_test, BATCH_SIZE, dict(), target_size = (image_height,image_width))
results = reconstructed_model.evaluate(test_gen, steps=len(df_test)/BATCH_SIZE)

print('Depth Loss', results[0])
print('Depth Acc', results[1])


In [ ]:
print(df_test.columns)

In [ ]:
for i in range(20):
    index = np.random.randint(1, len(df_test.index))
    img = cv2.imread(df_test['images'].iloc[index])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(image_height,image_width))
    img = img/255
    
    
    img = img[np.newaxis,:,:,:]
    predicted_img = reconstructed_model.predict(img)
#     predicted_img = cv2.cvtColor(predicted_img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(12,12))
    plt.subplot(1,3,1)
    plt.imshow(np.squeeze(img))
    plt.title('Original Image')
    
    plt.subplot(1,3,2)
    plt.imshow(np.squeeze(cv2.imread(df_test['masks'].iloc[index])))
    plt.title('Original Mask')
    
    plt.subplot(1,3,3)
    plt.imshow(np.squeeze(predicted_img) > 0.5)
    plt.title('Prediction')
    plt.show()
